<a href="https://colab.research.google.com/github/nbilasals/notebooks/blob/main/Cohort_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seattle ADU Building Permits - Cohort Analysis
## Urban Housing Insitute Research Project

**Research Question:** How do permit-to-completion timeliness vary based on permit issuence month? Are these seasonal patterns in contractor productivity?

**Dataset**: Seattle Building Permits (Public Open Data)

**Methodology**: Cohort analysis tracking completion rates at 3, 6, 9, and 12-month intervals

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width',None)
pd.set_option('display.precision',2)

In [ ]:
# Visualizations settings
sns.set_style('whitegrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

## 2. Load Seattle Building Permits Data

In [ ]:
# Load the dataset
df = pd.read_csv('/content/building-permits-current.csv')

In [ ]:
print("Seattle Building Permits - Dataset Overview: ")
print("Total records",len(df))
print("Total columns",len(df.columns))

Seattle Building Permits - Dataset Overview: 
Total records 59173
Total columns 20


In [ ]:
print("Column Summary:")
df.info()

Column Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59173 entries, 0 to 59172
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Application/Permit Number        59173 non-null  int64  
 1   Permit Type                      59173 non-null  object 
 2   Address                          58947 non-null  object 
 3   Description                      58901 non-null  object 
 4   Category                         55908 non-null  object 
 5   Action Type                      55968 non-null  object 
 6   Work Type                        59173 non-null  object 
 7   Value                            59173 non-null  float64
 8   Applicant Name                   58671 non-null  object 
 9   Application Date                 45305 non-null  object 
 10  Issue Date                       38346 non-null  object 
 11  Final Date                       26337 non-null  object 
 12  Ex

In [ ]:
print("First 5 Rows:")
display(df.head())

First 5 Rows:


,Application/Permit Number,Permit Type,Address,Description,Category,Action Type,Work Type,Value,Applicant Name,Application Date,Issue Date,Final Date,Expiration Date,Status,Contractor,Permit and Complaint Status URL,Master Use Permit,Latitude,Longitude,Location
0,6565996,Construction,890 NW ELFORD DR,Construct interior alterations to an existing ...,SINGLE FAMILY / DUPLEX,ADD/ALT,No plan review,50000.0,GREENSPACE CONSTRUCTION,2016-11-21T00:00:00,2016-11-21T00:00:00,NaN,2018-05-21T00:00:00,Permit Issued,GREENSPACE CONSTRUCTION,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.73,-122.37,"{'latitude': '47.72862996', 'human_address': '..."
1,6557481,Construction,10717 MIDVALE AVE N,Construct alterations to install a tank on an ...,COMMERCIAL,ADD/ALT,Plan Review,1500.0,JAMES QUINN COMPANY LLC,2016-11-01T00:00:00,2016-12-08T00:00:00,NaN,2018-06-08T00:00:00,Permit Issued,JAMES QUINN COMPANY LLC,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.71,-122.34,"{'latitude': '47.70761624', 'human_address': '..."
2,6560495,Construction,1101 NW 65TH ST,Construct interior kitchen remodel to existing...,SINGLE FAMILY / DUPLEX,ADD/ALT,No plan review,2000.0,ADAM,2016-10-13T00:00:00,2016-10-13T00:00:00,2016-12-29T00:00:00,2018-04-13T00:00:00,Permit Closed,ADAM,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.68,-122.37,"{'latitude': '47.67581843', 'human_address': '..."
3,6552155,Construction,1000 DEXTER AVE N,Blanket permit for interior non-structural alt...,COMMERCIAL,ALTER,Plan Review,100000.0,BLANKET:1000-1100 DEXTER AVE N,2016-08-18T00:00:00,2016-09-06T00:00:00,2016-10-14T00:00:00,2018-03-06T00:00:00,Permit Closed,BLANKET:1000-1100 DEXTER AVE N,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.63,-122.34,"{'latitude': '47.62817641', 'human_address': '..."
4,6551127,Construction,1101 NW 62ND ST,Construct alterations to kitchen in existing s...,SINGLE FAMILY / DUPLEX,ADD/ALT,No plan review,2500.0,STEVE KEENAN,2016-08-16T00:00:00,2016-08-16T00:00:00,2016-10-27T00:00:00,2018-02-16T00:00:00,Permit Closed,STEVE KEENAN,{'url': 'http://web6.seattle.gov/dpd/PermitSta...,NaN,47.67,-122.37,"{'latitude': '47.67363067', 'human_address': '..."


## Data Cleaning and Preparation
### Filter for ADU Single Family Resedential Family

In [ ]:
# Check unique permit types
print("Unique Permit Types:")
df['Permit Type'].value_counts()

Unique Permit Types:


,count
Permit Type,
Construction,52094
Demolition,4839
Site Development,2240


In [ ]:
# Filter for Single Family Resenditial permits
# Common varations in the dataset
adu_keywords = ['single family', 'residential','adu','construction']

# Create mask for relevant permits
mask = df['Permit Type'].str.lower().str.contains('|'.join(adu_keywords),na=False)
df_filtered=df[mask].copy()

In [ ]:
# Check categpries
print("Unique Permit Categories:")
df_filtered['Permit Type'].value_counts()

Unique Permit Categories:


,count
Permit Type,
Construction,52094


### Parse and Clean Date Columns

In [ ]:
# Parse Date
date_columns = ['Application Date','Issue Date','Final Date','Expiration Date']

for col in date_columns:
  if col in df_filtered.columns:
    df_filtered[col] = pd.to_datetime(df_filtered[col], errors='coerce')
    print(f"Parsed {col}")
  else:
    print(f"{col} not found in the DataFrame")

Parsed Application Date
Parsed Issue Date
Parsed Final Date
Parsed Expiration Date


In [ ]:
# Check date ranges
print("Date Ranges:")
for col in ['Application Date','Issue Date','Final Date']:
  if col in df_filtered.columns and df_filtered[col].notna().any():
    print(f"{col}: {df_filtered[col].min()} - {df_filtered[col].max()}")
  else:
    print(f"{col} not found in the DataFrame or contains no dates")

Date Ranges:
Application Date: 2000-10-17 00:00:00 - 2018-04-27 00:00:00
Issue Date: 2013-05-01 00:00:00 - 2018-04-27 00:00:00
Final Date: 2013-05-03 00:00:00 - 2018-04-27 00:00:00


### Filter for Last 24 Months and Required Fields

In [ ]:
# For cohort analysis, we MUST have Issue Date
df_with_issue = df_filtered[df_filtered['Issue Date'].notna()].copy()
print(f"Records with Issue Date: {len(df_with_issue):,}")

Records with Issue Date: 34,676


In [ ]:
# Get last 24 months from most recent Issue Date
max_date = df_with_issue['Issue Date'].max()
cutoff_date = max_date - pd.DateOffset(months=24)

print(f"Most recent Issue Date: {max_date.date()}")
print(f"24-month cutoff date: {cutoff_date.date()}")

Most recent Issue Date: 2018-04-27
24-month cutoff date: 2016-04-27


In [ ]:
# Filter for last 24 months
df_24mo = df_with_issue[df_with_issue['Issue Date']>=cutoff_date].copy()
print(f"Records in last 24 months: {len(df_24mo)}")

Records in last 24 months: 14742


In [ ]:
# Check completion status
df_24mo['Has_Final_Date'] = df_24mo['Final Date'].notna()
completed_count = df_24mo['Has_Final_Date'].sum()
in_progress_count = (-df_24mo['Has_Final_Date']).sum()

print(f"Completed Permits: {completed_count:,}")
print(f"In Progress Permits: {in_progress_count:,}")

Completed Permits: 7,037
In Progress Permits: 7,705


In [ ]:
# This is our cohort dataset
df_cohort = df_24mo.copy()

## Calculate Days to Completion

In [ ]:
# Calculate days from Issue to Final (only for completed permits)
df_cohort['Days_to_Complete'] = (df_cohort['Final Date'] - df_cohort['Issue Date']).dt.days

# Check for data quality issue
negative_days = (df_cohort['Days_to_Complete'] < 0).sum()
print(f"Negative Days to Complete: {negative_days}")

Negative Days to Complete: 0


In [ ]:
# Statistics for COMPLETED permits only
completed = df_cohort[df_cohort['Days_to_Complete'].notna()]

completed.describe().T

,count,mean,min,25%,50%,75%,max,std
Application/Permit Number,7037.0,6560204.19,6152679.0,6534505.0,6559830.0,6590309.0,6656490.0,43597.97
Value,7037.0,214305.66,0.0,10506.0,45000.0,150000.0,44878262.0,1137542.57
Application Date,7037,2016-11-27 19:48:42.665908736,2012-03-21 00:00:00,2016-06-22 00:00:00,2016-11-10 00:00:00,2017-05-08 00:00:00,2018-04-18 00:00:00,NaN
Issue Date,7037,2017-01-14 23:42:11.817535744,2016-04-27 00:00:00,2016-08-08 00:00:00,2016-12-22 00:00:00,2017-06-05 00:00:00,2018-04-18 00:00:00,NaN
Final Date,7037,2017-07-29 15:13:28.810572800,2016-05-02 00:00:00,2017-03-16 00:00:00,2017-08-24 00:00:00,2017-12-21 00:00:00,2018-04-27 00:00:00,NaN
Expiration Date,7037,2018-07-22 05:54:25.425607680,2017-10-27 00:00:00,2018-02-11 00:00:00,2018-07-04 00:00:00,2018-12-10 00:00:00,2019-10-28 00:00:00,NaN
Master Use Permit,340.0,3020675.3,3005586.0,3018597.75,3021854.0,3023130.0,3027060.0,3853.14
Latitude,7037.0,47.63,47.5,47.6,47.63,47.67,47.73,0.05
Longitude,7037.0,-122.33,-122.42,-122.36,-122.33,-122.31,-122.24,0.04
Days_to_Complete,7037.0,195.65,0.0,74.0,150.0,285.0,717.0,155.7


## Define Cohorts

In [ ]:
# Create cohort identifer: YYYY-MM Format
df_cohort['Cohort_Month'] = df_cohort['Issue Date'].dt.to_period('M')
df_cohort['Cohort_Month_str'] = df_cohort['Cohort_Month'].astype(str)


In [ ]:
# Count permits in each cohort
cohort_sizes = df_cohort.groupby('Cohort_Month_str').size().sort_index()
cohort_sizes

,0
Cohort_Month_str,
2016-04,78
2016-05,653
2016-06,843
2016-07,589
2016-08,685
2016-09,684
2016-10,583
2016-11,481
2016-12,467


In [ ]:
cohort_sizes.describe().T

,0
count,25.00
mean,589.68
std,137.32
min,78.00
25%,541.00
50%,589.00
75%,670.00
max,843.00


## Calculate Cohort Completion Status

In [ ]:
def calculate_completion_rate(cohort_df,months_threshold):
  days_threshold = months_threshold * 30

  # Count permits completed within threshold
  completed_in_time = (
      cohort_df['Days_to_Complete'].notna() & (cohort_df['Days_to_Complete'] <= days_threshold)
  ).sum()

  # Total permits in cohort
  total = len(cohort_df)

  # Percentage
  if total > 0 :
    return (completed_in_time / total) * 100
  else:
    return np.nan

In [ ]:
# Initialize cohort matrix
cohorts = sorted(df_cohort['Cohort_Month_str'].unique())
intervals = [3,6,9,12]

print(f"Calculating completion rates for {len(cohorts)} cohorts at {intervals} month intervals")

Calculating completion rates for 25 cohorts at [3, 6, 9, 12] month intervals


In [ ]:
cohort_matrix = pd.DataFrame(
    index=cohorts,
    columns =[f'{m}_months' for m in intervals]
)

In [ ]:
cohort_matrix

,3_months,6_months,9_months,12_months
2016-04,NaN,NaN,NaN,NaN
2016-05,NaN,NaN,NaN,NaN
2016-06,NaN,NaN,NaN,NaN
2016-07,NaN,NaN,NaN,NaN
2016-08,NaN,NaN,NaN,NaN
2016-09,NaN,NaN,NaN,NaN
2016-10,NaN,NaN,NaN,NaN
2016-11,NaN,NaN,NaN,NaN
2016-12,NaN,NaN,NaN,NaN
2017-01,NaN,NaN,NaN,NaN


In [ ]:
# Calculate for each cohort
for cohort in cohorts:
  cohort_data = df_cohort[df_cohort['Cohort_Month_str']==cohort]
  for months in intervals:
    rate = calculate_completion_rate(cohort_data,months)
    cohort_matrix.loc[cohort,f'{months}_months'] = rate

In [ ]:
# Convert to numeric
for col in cohort_matrix.columns:
  cohort_matrix[col] = pd.to_numeric(cohort_matrix[col],errors='coerce')

# Add cohort size column
cohort_matrix['Cohort_Size'] = cohort_sizes

# Reorder columns: Size first, then completion rates
cols = ['Cohort_Size'] + [f'{m}_months' for m in intervals]
cohort_matrix = cohort_matrix[cols]

In [ ]:
print("Cohort Completion Matrix (first 10 cohorts):")
cohort_matrix.head(10)

Cohort Completion Matrix (first 10 cohorts):


,Cohort_Size,3_months,6_months,9_months,12_months
2016-04,78,8.97,26.92,35.90,50.00
2016-05,653,15.16,34.00,44.26,54.36
2016-06,843,13.17,25.98,36.42,44.84
2016-07,589,16.64,31.58,42.78,50.08
2016-08,685,14.74,29.78,40.73,48.47
2016-09,684,16.67,29.24,40.50,50.58
2016-10,583,15.44,31.73,42.20,49.74
2016-11,481,14.97,29.94,41.37,48.44
2016-12,467,17.77,30.41,42.61,52.25
2017-01,534,18.91,36.14,45.69,55.24


## Identify Stalled Cohorts


In [ ]:
# Calculate average 6-month completion rate
avg_6mo = cohort_matrix['6_months'].mean()
std_6mo = cohort_matrix['6_months'].std()

print(f"Average 6-month completion rate: {avg_6mo:.2f}%")
print(f"Standard deviation of 6-month completion rate: {std_6mo:.2f}%")

Average 6-month completion rate: 27.37%
Standard deviation of 6-month completion rate: 8.57%


In [ ]:
# Define "stalled" as 1 standard deviation below average
stalled_threshold = avg_6mo - std_6mo
print(f"Stalled threshold: < {stalled_threshold:.1f}%")

Stalled threshold: < 18.8%


In [ ]:
# Flag stalled cohorts
cohort_matrix['Is_Stalled'] = cohort_matrix['6_months'] < stalled_threshold

stalled_cohorts = cohort_matrix[cohort_matrix['Is_Stalled']==True]

print(f"Stalled Cohorts Found: {len(stalled_cohorts)}")

Stalled Cohorts Found: 3


## Statistical AnalysisL Value vs Completion Tike

In [ ]:
# Get completed permits with valid values
analysis_df = completed[['Value','Days_to_Complete']].copy()

# Remove any outliers and zeros
analysis_df = analysis_df[
    (analysis_df['Value'] > 0) &
    (analysis_df['Value'] < analysis_df['Value'].quantile(0.99))&
    (analysis_df['Days_to_Complete'] < analysis_df['Days_to_Complete'].quantile(0.99))
]
analysis_df


,Value,Days_to_Complete
2,2.00e+03,77.0
3,1.00e+05,38.0
4,2.50e+03,72.0
6,6.00e+03,26.0
7,8.00e+03,51.0
9,9.02e+04,145.0
10,4.00e+04,178.0
11,9.80e+03,79.0
12,8.50e+04,43.0
13,4.20e+04,94.0


In [ ]:
# Calculate Pearson correlation
correlation, p_value = stats.pearsonr(analysis_df['Value'],analysis_df['Days_to_Complete'])

print(f"\nPearson Correlation Coefficient: {correlation:.2f}")
print(f"P-value: {p_value:.2f}")


Pearson Correlation Coefficient: 0.23
P-value: 0.00


In [ ]:
if p_value < 0.05:
  print("There is a statistically significant correlation between Value and Days to Complete.")
else:
  print("There is no statistically significant correlation between Value and Days to Complete.")

There is a statistically significant correlation between Value and Days to Complete.


## Seasonal Patterns

In [ ]:
# Define seasons
def get_season(month):
  if month in [12,1,2]:
    return 'Winter'
  elif month in [3,4,5]:
    return 'Spring'
  elif month in [6,7,8]:
    return 'Summer'
  else:
    return 'Fall'

df_cohort['Issue_Season'] = df_cohort['Issue Date'].dt.month.apply(get_season)

In [ ]:
# Calculate stats by season
seasonal_stats = df_cohort[df_cohort['Days_to_Complete'].notna()].groupby('Issue_Season').agg({'Days_to_Complete':['mean','median','count']})

print("Completion Time by Season (Permit Issued):")
print(seasonal_stats.round(1))

Completion Time by Season (Permit Issued):
             Days_to_Complete             
                         mean median count
Issue_Season                              
Fall                    192.9  146.0  1751
Spring                  196.1  165.0  1654
Summer                  222.0  175.0  2369
Winter                  149.4  105.0  1263


In [ ]:
# Calculate percentage difference from baseline (Summer)

if 'Summer' in seasonal_stats.index:
  summer_avg = seasonal_stats.loc['Summer',('Days_to_Complete','mean')]

  print("Seasonal Decay Analysis (vs Summer baseline): ")
  for season in seasonal_stats.index:
    season_avg = seasonal_stats.loc[season,('Days_to_Complete','mean')]
    diff = ((season_avg - summer_avg)/summer_avg)*100

    symbol = "🔴" if diff > 10 else "🟢" if diff <-10 else "🟡"
    print(f"{symbol} {season}: {season_avg:.0f} days {diff:+.1f}% vs Summer")

Seasonal Decay Analysis (vs Summer baseline): 
🟢 Fall: 193 days -13.1% vs Summer
🟢 Spring: 196 days -11.6% vs Summer
🟡 Summer: 222 days +0.0% vs Summer
🟢 Winter: 149 days -32.7% vs Summer


## Export Data for Looker Studio

In [ ]:
# Prepare main export dataset
looker_export = df_cohort[[
    'Application/Permit Number',
    'Permit Type',
    'Category',
    'Work Type',
    'Value',
    'Contractor',
    'Application Date',
    'Issue Date',
    'Final Date',
    'Expiration Date',
    'Status',
    'Days_to_Complete',
    'Cohort_Month_str',
    'Issue_Season',
    'Has_Final_Date',
    'Address',
    'Description'
]].copy()

KeyError: "['Is_Stalled'] not in index"

In [ ]:
# Add calculated fields for Looker
looker_export['Completion_Status'] = looker_export['Has_Final_Date'].map({True:'Completed',False:'In Progress'})
# Completion Time categories
looker_export['Completion_Category']= pd.cut(looker_export['Days_to_Complete'], bins=[0,90,180,270,365,np.inf], labels=['0-90','91-180','181-270','271-365','365+'])

# Value categories
looker_export['Value_Category']=pd.cut(looker_export['Value'],bins=[0,50000,100000,200000,300000,np.inf],labels=['0-500k','500k-1000k','1000k-2000k','2000k-3000k','3000k+'])

# Year and Month for easier filtering
looker_export['Issue_Year'] = looker_export['Issue Date'].dt.year
looker_export['Issue_Month_Name'] = looker_export['Issue Date'].dt.strftime('%B')
looker_export['Issue_Month_Num'] = looker_export['Issue Date'].dt.month

# Days since last issue (for tracking in-progress permits)
looker_export['Days_Since_Issue'] = (pd.Timestamp.now()-looker_export['Issue Date']).dt.days

In [ ]:
# Save main dataset
looker_export.to_csv('looker_export.csv',index=False)

In [ ]:
# Save cohort matrix (for reference)
cohort_matrix.to_csv('cohort_matrix.csv')

In [ ]:
## 14. Final Summary

print("\n" + "=" * 80)
print("COHORT ANALYSIS COMPLETE - SUMMARY")
print("=" * 80)

print(f"\nDataset:")
print(f"  Total permits analyzed: {len(df_cohort):,}")
print(f"  Completed: {completed_count:,} ({completed_count/len(df_cohort)*100:.1f}%)")
print(f"  In Progress: {in_progress_count:,} ({in_progress_count/len(df_cohort)*100:.1f}%)")

print(f"\nCohort Analysis:")
print(f"  Total cohorts: {len(cohort_matrix)}")
print(f"  Average cohort size: {cohort_sizes.mean():.1f} permits")
print(f"  Stalled cohorts identified: {len(stalled_cohorts)}")

print(f"\nCompletion Timeline:")
print(f"  Average: {completed['Days_to_Complete'].mean():.0f} days")
print(f"  Median: {completed['Days_to_Complete'].median():.0f} days")

print(f"\nKey Findings:")
print(f"  Value-Completion correlation: r = {correlation:.3f}")
print(f"  Seasonal variation: Detected (see analysis above)")
print(f"  6-month completion rate: {avg_6mo:.1f}% average")

print(f"\n📁 Files Created:")
print(f"  1. seattle_adu_looker_data.csv - Full dataset for Looker Studio")
print(f"  2. cohort_matrix.csv - Completion rates by cohort")
print(f"  3. analysis_summary.csv - Key metrics summary")

print(f"\n✅ NEXT STEPS:")
print(f"  1. Upload seattle_adu_looker_data.csv to Google Sheets")
print(f"  2. Connect to Looker Studio")
print(f"  3. Create visualizations:")
print(f"     - Cohort heatmap (pivot table)")
print(f"     - Timeline charts")
print(f"     - Distribution plots")
print(f"     - Scatter plots (Value vs Days)")
print(f"  4. Add interactive filters")
print(f"  5. Share dashboard with Urban Housing Institute")



COHORT ANALYSIS COMPLETE - SUMMARY

Dataset:
  Total permits analyzed: 14,742
  Completed: 7,037 (47.7%)
  In Progress: 7,705 (52.3%)

Cohort Analysis:
  Total cohorts: 25
  Average cohort size: 589.7 permits
  Stalled cohorts identified: 3

Completion Timeline:
  Average: 196 days
  Median: 150 days

Key Findings:
  Value-Completion correlation: r = 0.226
  Seasonal variation: Detected (see analysis above)
  6-month completion rate: 27.4% average

📁 Files Created:
  1. seattle_adu_looker_data.csv - Full dataset for Looker Studio
  2. cohort_matrix.csv - Completion rates by cohort
  3. analysis_summary.csv - Key metrics summary

✅ NEXT STEPS:
  1. Upload seattle_adu_looker_data.csv to Google Sheets
  2. Connect to Looker Studio
  3. Create visualizations:
     - Cohort heatmap (pivot table)
     - Timeline charts
     - Distribution plots
     - Scatter plots (Value vs Days)
  4. Add interactive filters
  5. Share dashboard with Urban Housing Institute
